In [1]:
from dotenv import load_dotenv
import os
from openai import AzureOpenAI
load_dotenv("credentials.env",override=True)

True

In [2]:
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
DEPLOYMENT_NAME = "gpt-4o"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
aoai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

client = AzureOpenAI(
    api_key = aoai_api_key,    
    api_version = aoai_api_version,
    azure_endpoint = aoai_endpoint
)


In [3]:
import openai
print(openai.__version__)

1.63.0


In [4]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Elon_Musk"  # Replace with the URL of the web page you want to scrape
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

if not os.path.exists('data'): 
    os.makedirs('data')

if not os.path.exists('data/elon.md'):
    elon = soup.text.split('\nSee also')[0]
    with open('data/elon.md', 'w', encoding='utf-8') as f:
        f.write(elon)
else:
    with open('data/elon.md', 'r') as f:
        elon = f.read()

In [5]:
import yaml

if os.path.exists('data/graphrag'):
    !python -m graphrag.index --init --root data/graphrag


⠋ GraphRAG Indexer 
Initializing project at data/graphrag
⠋ GraphRAG Indexer 
Traceback (most recent call last):
⠋ GraphRAG Indexer 
  File "<frozen runpy>", line 198, in _run_module_as_main
⠋ GraphRAG Indexer 
  File "<frozen runpy>", line 88, in _run_code
⠋ GraphRAG Indexer 
  File 
"c:\Users\nileshjoshi\git\aoai-poc-hub\common\envs\aoai_poc_e6\Lib\site-package
s\graphrag\index\__main__.py", line 104, in <module>
⠋ GraphRAG Indexer 
    index_cli(
⠋ GraphRAG Indexer 
  File 
"c:\Users\nileshjoshi\git\aoai-poc-hub\common\envs\aoai_poc_e6\Lib\site-package
s\graphrag\index\cli.py", line 126, in index_cli
⠋ GraphRAG Indexer 
    _initialize_project_at(root_dir, progress_reporter)
⠋ GraphRAG Indexer 
  File 
"c:\Users\nileshjoshi\git\aoai-poc-hub\common\envs\aoai_poc_e6\Lib\site-package
s\graphrag\index\cli.py", line 199, in _initialize_project_at
⠋ GraphRAG Indexer 
    raise ValueError(msg)
⠋ GraphRAG Indexer 
ValueError: Project already initialized at data\graphrag
⠋ GraphRAG Indexer 


In [6]:

with open('data/graphrag/settings.yaml', 'r') as f:
    settings_yaml = yaml.load(f, Loader=yaml.FullLoader)
settings_yaml['llm']['model'] = "gpt-4o"
settings_yaml['llm']['api_key'] = aoai_api_key 
settings_yaml['llm']['type'] = 'azure_openai_chat' 
settings_yaml['embeddings']['llm']['api_key'] = aoai_api_key
settings_yaml['embeddings']['llm']['type'] = 'azure_openai_embedding'
settings_yaml['embeddings']['llm']['model'] = EMBEDDING_MODEL_NAME 
settings_yaml['llm']['api_version'] = aoai_api_version
settings_yaml['llm']['deployment_name'] = DEPLOYMENT_NAME
settings_yaml['llm']['api_base'] = aoai_endpoint
settings_yaml['embeddings']['llm']['api_version'] = aoai_api_version
settings_yaml['embeddings']['llm']['deployment_name'] = EMBEDDING_MODEL_NAME
settings_yaml['embeddings']['llm']['api_base'] = aoai_endpoint

with open('data/graphrag/settings.yaml', 'w') as f:
    yaml.dump(settings_yaml, f)

#if not os.path.exists('data/graphrag/input'):
    #os.makedirs('data/graphrag/input')
    #!copy data/elon.md data/graphrag/input/elon.txt
!python -m graphrag.index --root ./data/graphrag

⠋ GraphRAG Indexer 
Logging enabled at 
C:\Users\nileshjoshi\git\ai-lab\data\graphrag\output\indexing-engine.log
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
Traceback (most recent call last):
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 0 files loaded (1 filtered) - 100%  0…
  File "<frozen runpy>", line 198, in _run_module_as_main
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 0 files loaded (1 filtered) - 100%  0…
  File "<frozen runpy>", line 88, in _run_code
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 0 files loaded (1 filtered) - 100%  0…
  File 
"c:\Users\nileshjoshi\git\ai-lab\.conda\Lib\site-packages\graphrag\index\__main
__.py", line 104, in <module>
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 0 files loaded (1 filtered) - 100%  0…
    index_cli(
⠙ GraphRAG Indexer 
└── Loading Input (InputFileType.text) - 0 files loaded (1 filtered) - 100%  0…
  File 
"c:\Users\nileshjoshi\git\ai-lab\.conda\Lib\site-packages\graphrag

In [6]:
import subprocess
import re
DEFAULT_RESPONSE_TYPE = 'Summarize and explain in 1-2 paragraphs with bullet points using at most 300 tokens'
DEFAULT_MAX_CONTEXT_TOKENS = 10000

def remove_data(text):
  return re.sub(r'\[Data:.*?\]', '', text).strip()


def ask_graph(query,method):
  env = os.environ.copy() | {
    'GRAPHRAG_GLOBAL_SEARCH_MAX_TOKENS': str(DEFAULT_MAX_CONTEXT_TOKENS),
  }
  command = [
    'python', '-m', 'graphrag.query',
    '--root', './data/graphrag',
    '--method', method,
    '--response_type', DEFAULT_RESPONSE_TYPE,
    query,
  ]
  output = subprocess.check_output(command, universal_newlines=True, env=env, stderr=subprocess.DEVNULL)
  return remove_data(output.split('Search Response: ')[0])

GrpahRag offers 2 types of search:

Global Search for reasoning about holistic questions about the corpus by leveraging the community summaries.
Local Search for reasoning about specific entities by fanning-out to their neighbors and associated concepts.

In [7]:
from IPython.display import Markdown
global_query="where did Elon Musk studied"
global_result = ask_graph(global_query,'global')

Markdown(global_result)

creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
### Elon Musk's Educational Background

- **University of Pennsylvania**: Elon Musk earned degrees in economics and physics from the University of Pennsylvania. This educational background provided a strong foundation for his later ventures in technology and business .
- **Stanford University**: Musk briefly attended Stanford University, which contributed to his early career development before he left to pursue entrepreneurial opportunities .

In [8]:
from IPython.display import Markdown
local_query="What and how many companies and subsidieries founded by Elon Musk"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
Elon Musk has founded and co-founded several high-profile companies and subsidiaries across various industries. Here is a summary of the key companies and subsidiaries he has been involved with:

### Companies Founded or Co-Founded by Elon Musk
- **Zip2**: Co-founded with his brother Kimbal Musk, an online city guide software company .
- **X.com (PayPal)**: Co-founded as an online payment company, which later became PayPal .
- **SpaceX**: Founded in 2002, specializing in spaceflight services .
- **Tesla, Inc.**: Early investor and later CEO, product architect, and chairman .
- **SolarCity**: Helped create in 2006, later acquired by Tesla .
- **The Boring Company**: Founded in 2016, focusing on tunnel construction .
- **Neuralink**: Co-founded in 2016, developing brain-machine interfaces .
- **OpenAI**: Co-founded in 2015, focusing on artificial intelligence research .
- **xAI**: Founded in 2023, an artificial intelligence company .

### Subsidiaries
- **Tesla Energy**: A subsidiary of Tesla, Inc., formed after the acquisition of SolarCity .
- **Starlink**: Operated by SpaceX, providing satellite internet services .

### Summary
- **Total Companies Founded or Co-Founded**: 9
- **Total Subsidiaries**: 2

Elon Musk's ventures span various fields including technology, transportation, space exploration, and renewable energy, showcasing his diverse interests and innovative contributions .

In [9]:
global_query="What are the major accomplishments of Elon Musk?"
global_result = ask_graph(global_query,'global')

Markdown(global_result)

creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
### Major Accomplishments of Elon Musk

Elon Musk has achieved significant milestones across various industries through his leadership and entrepreneurial ventures:

- **Tesla, Inc.**: Co-founded Tesla, providing initial financing and serving as CEO and product architect. Under his leadership, Tesla has become a major player in the electric vehicle and clean energy markets .
- **SpaceX**: Founded SpaceX in 2002 with $100 million from the sale of PayPal. SpaceX has become a major player in the aerospace industry, achieving significant milestones in spaceflight services and reusable rocket technology .
- **PayPal**: Co-founded X.com in 1999, which later merged with Confinity to form PayPal. PayPal became a leading online payment system and was acquired by eBay for $1.5 billion in 2002 .
- **X Corp. (formerly Twitter)**: Acquired Twitter in 2022 for $44 billion and rebranded it as X Corp. in 2023. Holds multiple key positions, including owner, CTO, and executive chairman, significantly influencing the platform's direction and policies .
- **Other Ventures**: Leads other high-profile companies such as Neuralink and The Boring Company, positioning himself as a key figure in neurotechnology and infrastructure development .

These accomplishments highlight Musk's influence in advancing technology and innovation across multiple sectors.

In [11]:
from IPython.display import Markdown
local_query="Summarize business career of Elon Musk"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
## Elon Musk's Business Career

Elon Musk has had a highly influential and diverse business career, marked by his leadership in multiple high-profile companies and innovative ventures:

- **Zip2 and PayPal**: Musk co-founded Zip2 with his brother Kimbal, which was sold to Compaq for $307 million. He then co-founded X.com, which became PayPal and was acquired by eBay for $1.5 billion .
- **SpaceX**: Founded in 2002 using $100 million from the PayPal sale, SpaceX focuses on spaceflight services and has become a leader in aerospace manufacturing .
- **Tesla, Inc.**: Musk joined Tesla as an early investor in 2004, later becoming CEO and product architect. Tesla is a leading electric vehicle and clean energy company .
- **SolarCity**: Created with Musk's help in 2006, SolarCity was acquired by Tesla in 2016 and became Tesla Energy .
- **Neuralink and OpenAI**: Co-founded Neuralink in 2016 to develop brain-machine interfaces and OpenAI in 2015 to advance digital intelligence .
- **The Boring Company and Hyperloop**: Founded The Boring Company in 2016 for tunnel construction and conceptualized the Hyperloop high-speed transportation system .
- **Twitter/X**: Acquired Twitter in 2022 for $44 billion, rebranding it as X in 2023 .
- **xAI**: Founded xAI in 2023 to focus on artificial intelligence .

Musk's ventures have significantly impacted technology, transportation, and space exploration, solidifying his reputation as a leading figure in innovation .

In [12]:
from IPython.display import Markdown
local_query="list all relatives of Elon Musk"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
Elon Musk has several notable relatives who have played various roles in his life and career. Here is a list of his known relatives:

### Immediate Family
- **Maye Musk**: Elon Musk's mother, a Canadian-born model and dietitian .
- **Errol Musk**: Elon Musk's father .
- **Kimbal Musk**: Elon Musk's brother, co-founder of Zip2 .
- **Tosca Musk**: Elon Musk's sister .

### Extended Family
- **Lyndon Rive**: Elon Musk's cousin .

### Marriages
- **Justine Wilson**: Elon Musk's first wife .
- **Talulah Riley**: Elon Musk's second wife, married twice .

These relationships have influenced various aspects of Musk's personal and professional life.

In [13]:
from IPython.display import Markdown
local_query="list all peoples associated with Elon Musk with their relation to him"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
### People Associated with Elon Musk

- **Maye Musk**: Mother of Elon Musk, through whom he acquired Canadian citizenship .
- **Errol Musk**: Father of Elon Musk .
- **Kimbal Musk**: Brother of Elon Musk and co-founder of Zip2 .
- **Tosca Musk**: Sister of Elon Musk .
- **Lyndon Rive**: Cousin of Elon Musk .
- **Talulah Riley**: Second wife of Elon Musk, married twice .
- **Justine Wilson**: First wife of Elon Musk .
- **Donald Trump**: US presidential candidate in the 2024 election, supported by Elon Musk .
- **Vivek Ramaswamy**: Co-leader of the Department of Government Efficiency (DOGE) advisory board with Elon Musk .

These individuals have played various roles in Elon Musk's personal and professional life, contributing to his career and public image.

In [14]:
from IPython.display import Markdown
global_query="list all peoples associated with Elon Musk with their relation to him"
global_result = ask_graph(local_query,'global')

Markdown(global_result)

creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
### People Associated with Elon Musk

- **Maye Musk**
  - Relation: Mother
  - Involvement: Various ventures with Elon Musk .

- **Kimbal Musk**
  - Relation: Brother
  - Involvement: Collaborated on multiple ventures with Elon Musk .

- **Lyndon Rive**
  - Relation: Cousin
  - Involvement: Various ventures with Elon Musk .

- **Donald Trump**
  - Relation: Political Ally
  - Involvement: Received vocal and financial support from Elon Musk during the 2024 US presidential election; appointed Musk as co-leader of the Department of Government Efficiency (DOGE) advisory board .

- **Vivek Ramaswamy**
  - Relation: Co-leader of DOGE advisory board
  - Involvement: Co-leads the Department of Government Efficiency (DOGE) advisory board alongside Elon Musk .

In [15]:
from IPython.display import Markdown
local_query="provide 10 points summary of Musk's life?"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
### Summary of Elon Musk's Life

- **Early Life and Education**:
  - Born in Pretoria, South Africa, Musk briefly attended the University of Pretoria before immigrating to Canada at age 18. He later attended Queen's University and the University of Pennsylvania, earning degrees in economics and physics .

- **Founding Zip2 and PayPal**:
  - Co-founded Zip2 with his brother Kimbal, which was sold to Compaq for $307 million. Later co-founded X.com, which became PayPal and was acquired by eBay for $1.5 billion .

- **SpaceX**:
  - Founded SpaceX in 2002 using $100 million from the sale of PayPal. SpaceX focuses on spaceflight services and has become a leader in aerospace manufacturing .

- **Tesla, Inc.**:
  - Early investor in Tesla, Inc., later becoming CEO and product architect. Tesla is a leading electric vehicle and clean energy company .

- **SolarCity and Tesla Energy**:
  - Helped create SolarCity, which was acquired by Tesla and became Tesla Energy, focusing on solar energy solutions .

- **Neuralink and OpenAI**:
  - Co-founded Neuralink and OpenAI, focusing on neurotechnology and artificial intelligence, respectively .

- **The Boring Company and Hyperloop**:
  - Founded The Boring Company to develop tunnel construction and conceptualized the Hyperloop high-speed transportation system .

- **Acquisition of Twitter**:
  - Acquired Twitter in 2022 for $44 billion, rebranding it as X and integrating it into X Corp. .

- **Legal and Regulatory Challenges**:
  - Faced significant legal issues, including a lawsuit from the SEC in 2018 over false statements about Tesla's privatization .

- **Philanthropy and Political Involvement**:
  - Founded the Musk Foundation, supporting various scientific and educational causes. Became a vocal supporter of Donald Trump in the 2024 US presidential election .

These points provide a comprehensive overview of Elon Musk's life, highlighting his major ventures, educational background, and significant challenges.

In [7]:
from IPython.display import Markdown
local_query="Describe the childhood of Elon Musk?"
local_result = ask_graph(local_query,'global')

Markdown(local_result)

creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
I am sorry but I am unable to answer this question given the provided data.

In [8]:
from IPython.display import Markdown
local_query="Describe the childhood of Elon Musk?"
local_result = ask_graph(local_query,'local')

Markdown(local_result)

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o', 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-ada-002', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://njo-npd-openai-dev-ce.openai.azure.com/', 'api_version': '2024-08-01-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-ada-002', 'model_supports_json': None, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
### Elon Musk's Childhood

- **Birth and Early Life**: Elon Musk was born on June 28, 1971, in Pretoria, South Africa .
- **Family Background**: He is a member of the wealthy Musk family. His father, Errol Musk, is an engineer, and his mother, Maye Musk, is a Canadian-born model and dietitian .
- **Education**: Musk briefly attended the University of Pretoria before immigrating to Canada at the age of 18 .
- **Immigration to Canada**: He acquired Canadian citizenship through his mother and later attended Queen's University in Canada .

Elon Musk's early life was marked by his international background and the influence of his family's wealth and professional achievements.

#### ----- Differences between RAG and GraphRAG

Retrieval Mechanism
RAG: Utilizes embeddings and vector databases for information retrieval.
GraphRAG: Employs graph structures and graph databases to manage data.

Contextual Understanding
RAG: Lacks a comprehensive understanding of data, relying primarily on similarity scores.
GraphRAG: Offers enhanced overall data understanding by capturing associated information and related entities.

Application
RAG: Best suited for general question-answering and straightforward information retrieval tasks.
GraphRAG: More effective for complex use cases that involve structured knowledge and intricate relationships.

Complexity
RAG: Easier to implement and deploy.
GraphRAG: More complicated, necessitating knowledge graph maintenance and management.

Scalability
RAG: Highly scalable and adaptable.
GraphRAG: More complex to scale, requiring modifications to the graph structure.

Maintenance
RAG: Low maintenance with straightforward updates.
GraphRAG: Higher maintenance demands due to the complexities of graph structures.

Integration
RAG: Easier to integrate into existing systems.
GraphRAG: More complicated integration, involving management of graph databases.

Learning Curve
RAG: Easier to grasp with basic knowledge of vectors and embeddings.
GraphRAG: Requires additional expertise in graph analytics and algorithms.

Cost
RAG: Generally incurs lower costs due to simpler infrastructure.
GraphRAG: Higher costs associated with increased computational requirements and knowledge graph maintenance.



RAG -- 
    Advantages: Simple implementation, Faster retrieval times

    Limitations: Less effective at capturing complex relationships among data.

GraphRAG -- 
    Advantages: Superior at capturing complex relationships, Leverages data connections effectively
    
    Limitations: Computationally intensive, Requires ongoing graph management.